In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sqlalchemy import create_engine

In [3]:
path="F:\\DOUTORADO_RENATO\\PESQUISAS\\MEUS_TRABALHOS\\AGUA_E_SAL\\data"
folder="01_05"

dados_excel = []
all_data_frames = []

In [ ]:
for rodadas in os.listdir(os.path.join(path,folder)):
    new_path = os.path.join(path,folder,rodadas)
    
    
    for rodada in os.listdir(new_path):
        amostras = os.path.join(new_path, rodada)
    
    
        if os.path.isdir(amostras):
            file = os.listdir(amostras)
            file = [os.path.join(amostras, x) for x in file]

            for files in file:
                df = pd.read_csv(files, sep='\t', decimal=',', header=None)
                df.columns = ['wavelength', 'intensity measurement LPG', 'intensity reference LPG', 'noise 1', 'noise 2']
                df.drop(columns=['noise 1', 'noise 2'], inplace=True)
                df['rodadas'] = os.path.basename(rodadas)
                df['amostras'] = os.path.basename(amostras)
                
                all_data_frames.append(df)

In [5]:
big_df = pd.concat(all_data_frames, ignore_index=True)
big_df['dia'] = os.path.basename(folder)

In [6]:
output_path = os.path.join(path, f"{folder}_todos_espectros.db")
engine = create_engine(f"sqlite:///{output_path}")
big_df.to_sql(
    name="spectra",
    con=engine,
    if_exists="replace",   # ou 'append' se preferir acrescentar a cada execução
    index=False,
    chunksize=100_000      # opcional, para não carregar tudo na memória do SQLite de uma vez
)

print(f"Banco SQLite gravado em: {output_path}")

Banco SQLite gravado em: F:\DOUTORADO_RENATO\PESQUISAS\MEUS_TRABALHOS\AGUA_E_SAL\data\01_05_todos_espectros.db


In [7]:
df_teste = pd.read_sql("SELECT * FROM spectra", engine)

In [8]:
df_teste.head()

,wavelength,intensity measurement LPG,intensity reference LPG,rodadas,amostras,dia
0,1500.000,-14.9851,-21.1195,RODADA_1,A_0,01_05
1,1500.005,-14.9882,-21.1167,RODADA_1,A_0,01_05
2,1500.010,-14.9898,-21.1146,RODADA_1,A_0,01_05
3,1500.015,-14.9872,-21.1109,RODADA_1,A_0,01_05
4,1500.020,-14.9897,-21.1068,RODADA_1,A_0,01_05
